In [1]:
import numpy as np
import pandas as pd
import gc
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import math
from lightgbm.sklearn import LGBMClassifier
from collections import Counter  
import time
from scipy.stats import kurtosis,iqr
from scipy import ptp
from tqdm import tqdm
import datetime
from sklearn.metrics import accuracy_score, roc_auc_score,log_loss,f1_score
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings
import os
from sklearn.externals import joblib
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [34]:
### 北京市重点区域信息
cols = ['id', 'area_name', 'area_type', 'Center_x', 'Center_y', 'Grid_x', 'Grid_y', 'area']
area_passenger_info = pd.read_csv('/home/poac/AnomalyDetectionDataset/Crowd_density_prediction/area_passenger_info.csv', names=cols)

In [35]:
area_passenger_info.iloc[0:5]

,id,area_name,area_type,Center_x,Center_y,Grid_x,Grid_y,area
0,1,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
1,2,古塔公园,旅游景点;公园,116.535404,39.878676,116.535232,39.878140,575268.839844
2,3,中央戏剧学院,教育培训;高等院校,116.404667,39.935999,116.404148,39.936165,17024.636719
3,4,丰联广场,购物;购物中心,116.438556,39.923713,116.438117,39.924203,11695.019531
4,5,中华女子学院,教育培训;高等院校,116.435352,39.994935,116.434508,39.994636,83053.546875


In [38]:
### 重点区域人流量情况
cols = ['id', 'date/hour', 'index']
area_passenger_index = pd.read_csv('/home/poac/AnomalyDetectionDataset/Crowd_density_prediction/area_passenger_index.csv', names=cols)
area_passenger_index['date/hour'] = area_passenger_index['date/hour'].apply(lambda x: str(x)[0:4]+'-'+str(x)[4:6]+'-'+str(x)[6:8]+'-'+str(x)[8:])
area_passenger_index['date/hour'] = pd.to_datetime(area_passenger_index['date/hour'])
area_passenger_index.shape

(717840, 3)

In [39]:
area_passenger_index.iloc[0:5]

,id,date/hour,index
0,1,2020-01-17 00:00:00,1.8
1,1,2020-01-17 01:00:00,1.5
2,1,2020-01-17 02:00:00,1.3
3,1,2020-01-17 03:00:00,1.3
4,1,2020-01-17 04:00:00,1.7


In [40]:
### 将北京市重点区域信息和重点区域人流量合并
area_passenger_index = area_passenger_index.merge(area_passenger_info,how = 'left',on = ['id'])
area_passenger_index.shape

(717840, 10)

In [41]:
area_passenger_index.iloc[0:20]

,id,date/hour,index,area_name,area_type,Center_x,Center_y,Grid_x,Grid_y,area
0,1,2020-01-17 00:00:00,1.8,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
1,1,2020-01-17 01:00:00,1.5,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
2,1,2020-01-17 02:00:00,1.3,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
3,1,2020-01-17 03:00:00,1.3,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
4,1,2020-01-17 04:00:00,1.7,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
5,1,2020-01-17 05:00:00,1.2,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
6,1,2020-01-17 06:00:00,2.9,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
7,1,2020-01-17 07:00:00,4.7,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
8,1,2020-01-17 08:00:00,6.1,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
9,1,2020-01-17 09:00:00,5.2,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
